# Make All Tables for Main Manuscript

Gaylen Fronk  
June 4, 2024

In [ ]:
#| message: false
#| warning: false

# packages for script
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

ℹ SHA-1 hash of file is "c045eee2655a18dc85e715b78182f176327358a7"

In [ ]:
#| message: false
#| warning: false

# handle conflicts
options(conflicts.policy = "depends.ok")

In [ ]:
# absolute paths
switch (Sys.info()[['sysname']],
        # PC paths
        Windows = {
          path_data <- str_c("P:/studydata/match/processed_data/")
          path_models <- str_c("P:/studydata/match/models/pp_hybrid_wk4_outcome")},
        
        # IOS paths
        Darwin = {
          path_data <- str_c("/Volumes/private/studydata/match/processed_data/")
          path_models <- str_c("/Volumes/private/studydata/match/models/pp_hybrid_wk4_outcome")},
        
        # Linux paths
        Linux = {
          path_data <- str_c("~/mnt/private/studydata/match/processed_data/")
          path_models <- str_c("~/mnt/private/studydata/match/models/pp_hybrid_wk4_outcome")}
)

In [ ]:
# chunk defaults

options(tibble.width = Inf)
options(tibble.print_max = Inf)

## Tables 1, 2, & 3 (Features & Sample Characteristics)

In [ ]:
d <- read_csv(file.path(path_data, "match_cln.csv"),
              show_col_types = FALSE)

### Table 1: Feature Categories

Set up data

In [ ]:
dt3 <- d |> select(-contains("outcome"), -subid)

Build table

In [ ]:
t1_feat_misc <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Treatment", "Incarcerated", "Works Third Shift"),
  type = c("Categorical", "Dichotomous", "Dichotomous"),
  n_feat = c(1, 1, 1)
) |> 
  add_row(feat_cat = "Miscellaneous",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat_psych_sud <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Frequency of Drinking Alcohol", "Quantity of Alcohol", "Binge Drinking", "Short Inventory of Problems-2 (Revised) [@kilukShortInventoryProblems2013]", "Life Satisfaction", "Life Enjoyment", "Psychological Disorder Diagnoses", "Positive and Negative Affect Schedule [@crawfordPositiveNegativeAffect2004a]", "Snaith-Hamilton Pleasure Scale [@snaithScaleAssessmentHedonic1995a]", "Anxiety Severity Index-3 [@taylorRobustDimensionsAnxiety2007a]", "Distress Tolerance Scale [@simonsDistressToleranceScale2005a]", "Patient History Questionnaire-9 [@kroenkePHQ9ValidityBrief2001a]"),
  type = c("Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Dichotomous", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Ordinal"),
  n_feat = c(1, 1, 1, 16, 1, 1, 7, 8, 15, 22, 20, 11)
) |> 
  add_row(feat_cat = "Mental Health & Psychological Traits",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat_smoking <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Baseline Carbon Monoxide", "Carbon Monoxide Exposure", "Age of 1st Cigarette", "Age Became Daily Smoker", "Years Smoking", "Cigarettes Per Day (Heaviest)", "Use of Other Tobacco Products", "Number of Previous Quit Attempts", "Last Recent Quit Attempt", "Longest Quit Attempt", "Previous Quit Methods Used", "Cigarettes Per Day (Current)", "Motivation to Quit", "Self-Efficacy for Quitting in Next 30 Days", "Confidence to Quit", "Importance to Quit", "DSM5 Tobacco Use Disorder [@americanpsychiatricassociationDiagnosticStatisticalManual2013c]", "Fagerstrom Test of Nicotine Dependence [@heathertonFagerstromTestNicotine1991]", "Wisconsin Inventory of Smoking Dependence Motives-37 [@smithDevelopmentBriefWisconsin2010]", "Smoke Menthol Cigarettes", "Wisconsin Smoking Withdrawal Scale-2 [@smithRevisionWisconsinSmoking2021]"),
  type = c("Numeric", "Dichotomous", "Numeric", "Numeric", "Numeric", "Numeric", "Categorical (Unordered)", "Numeric", "Ordinal", "Ordinal", "Dichotomous", "Numeric", "Ordinal", "Ordinal", "Ordinal", "Ordinal", "Dichotomous", "Dichotomous, Ordinal", "Ordinal", "Categorical (Unordered)", "Ordinal"),
  n_feat = c(1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 6, 1, 1, 1, 1, 1, 14, 6, 49, 1, 45)
) |> 
  add_row(feat_cat = "Smoking Use & History",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat_medical <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Diabetes Diagnosis", "Multidimensional Fatigue Inventory [@smetsMultidimensionalFatigueInventory1995]", "Berlin Sleep Questionnaire [@netzerUsingBerlinQuestionnaire1999]", "Body Mass Index", "Health-related Quality of Life Scale [@taylorMeasuringHealthyDays2000]", "Healthy Days Symptoms Module [@moriartyCDCStudiesCommunity1996]"),
  type = c("Dichotomous", "Ordinal", "Ordinal, Dichotomous", "Numeric", "Ordinal, Numeric", "Numeric"),
  n_feat = c(1, 26, 3, 1, 4, 5)
) |> 
  add_row(feat_cat = "Medical & Physical Health",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat_social_environ <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Spouse Smokes", "Live with Another Smoker", "People Close to You Who Smoke", "Ban on Smoking at Home", "Ban on Smoking at Work", "Time Around Other Smokers"),
  type = c("Categorical (unordered)", "Categorical (unordered)", "Categorical (unordered)", "Dichotomous", "Categorical (unordered)", "Ordinal"),
  n_feat = c(1, 1, 5, 1, 1, 2)
) |> 
  add_row(feat_cat = "Social & Environmental Characteristics",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat_demo <- tibble(
  feat_cat = NA_character_,
  feat_name = c("Gender", "Age", "Race", "Marital Status", "Income", "Ethnicity", "Employment"),
  type = c("Categorical (unordered)", "Numeric", "Categorical (unordered)", "Categorical (unordered)", "Ordinal", "Categorical (Unordered)", "Categorical (unordered)"),
  n_feat = c(1, 1, 1, 1, 1, 1, 1)
) |> 
  add_row(feat_cat = "Demographic Characteristics",
  feat_name = NA_character_,
  type = c(),
  n_feat = NA_real_,
  .before = 1)

t1_feat <- bind_rows(t1_feat_demo, t1_feat_smoking) |> 
  bind_rows(t1_feat_social_environ) |> 
  bind_rows(t1_feat_psych_sud) |> 
  bind_rows(t1_feat_medical) |> 
  bind_rows(t1_feat_misc) |> 
  rename(Category = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `Number of Features` = n_feat)

Table 1

In [ ]:
#| label: tbl-featureshtml
#| tbl-cap: "Individual Differences"

t1_feat |>
  mutate(across(where(is.numeric), as.character)) |>
  mutate(across(everything(), ~replace_na(., "")))  |>
  knitr::kable() 

  ---------------------------------------------------------------------------------------------------------------
  Category          Feature Name                                                        Type           Number of
                                                                                                       Features
  ----------------- ------------------------------------------------------------------- -------------- ----------
  Demographic                                                                                          
  Characteristics                                                                                      

                    Gender                                                              Categorical    1
                                                                                        (unordered)    

                    Age                                                                 Numeric        1

                    Race                                                                Categorical    1
                                                                                        (unordered)    

                    Marital Status                                                      Categorical    1
                                                                                        (unordered)    

                    Income                                                              Ordinal        1

                    Ethnicity                                                           Categorical    1
                                                                                        (Unordered)    

                    Employment                                                          Categorical    1
                                                                                        (unordered)    

  Smoking Use &                                                                                        
  History                                                                                              

                    Baseline Carbon Monoxide                                            Numeric        1

                    Carbon Monoxide Exposure                                            Dichotomous    1

                    Age of 1st Cigarette                                                Numeric        1

                    Age Became Daily Smoker                                             Numeric        1

                    Years Smoking                                                       Numeric        1

                    Cigarettes Per Day (Heaviest)                                       Numeric        1

                    Use of Other Tobacco Products                                       Categorical    5
                                                                                        (Unordered)    

                    Number of Previous Quit Attempts                                    Numeric        1

                    Last Recent Quit Attempt                                            Ordinal        1

                    Longest Quit Attempt                                                Ordinal        1

                    Previous Quit Methods Used                                          Dichotomous    6

                    Cigarettes Per Day (Current)                                        Numeric        1

                    Motivation to Quit                                                  Ordinal        1

                    Self-Efficacy for Quitting in Next 30 Days                          Ordinal        1

                    Confidence to Quit                                                  Ordinal        1

                    Importance to Quit                                                  Ordinal        1

                    DSM5 Tobacco Use Disorder                                           Dichotomous    14
                    [@americanpsychiatricassociationDiagnosticStatisticalManual2013c]                  

                    Fagerstrom Test of Nicotine Dependence                              Dichotomous,   6
                    [@heathertonFagerstromTestNicotine1991]                             Ordinal        

                    Wisconsin Inventory of Smoking Dependence Motives-37                Ordinal        49
                    [@smithDevelopmentBriefWisconsin2010]                                              

                    Smoke Menthol Cigarettes                                            Categorical    1
                                                                                        (Unordered)    

                    Wisconsin Smoking Withdrawal Scale-2                                Ordinal        45
                    [@smithRevisionWisconsinSmoking2021]                                               

  Social &                                                                                             
  Environmental                                                                                        
  Characteristics                                                                                      

                    Spouse Smokes                                                       Categorical    1
                                                                                        (unordered)    

                    Live with Another Smoker                                            Categorical    1
                                                                                        (unordered)    

                    People Close to You Who Smoke                                       Categorical    5
                                                                                        (unordered)    

                    Ban on Smoking at Home                                              Dichotomous    1

                    Ban on Smoking at Work                                              Categorical    1
                                                                                        (unordered)    

                    Time Around Other Smokers                                           Ordinal        2

  Mental Health &                                                                                      
  Psychological                                                                                        
  Traits                                                                                               

                    Frequency of Drinking Alcohol                                       Ordinal        1

                    Quantity of Alcohol                                                 Ordinal        1

                    Binge Drinking                                                      Ordinal        1

                    Short Inventory of Problems-2 (Revised)                             Ordinal        16
                    [@kilukShortInventoryProblems2013]                                                 

                    Life Satisfaction                                                   Ordinal        1

                    Life Enjoyment                                                      Ordinal        1

                    Psychological Disorder Diagnoses                                    Dichotomous    7

                    Positive and Negative Affect Schedule                               Ordinal        8
                    [@crawfordPositiveNegativeAffect2004a]                                             

                    Snaith-Hamilton Pleasure Scale [@snaithScaleAssessmentHedonic1995a] Ordinal        15

                    Anxiety Severity Index-3 [@taylorRobustDimensionsAnxiety2007a]      Ordinal        22

                    Distress Tolerance Scale [@simonsDistressToleranceScale2005a]       Ordinal        20

                    Patient History Questionnaire-9 [@kroenkePHQ9ValidityBrief2001a]    Ordinal        11

  Medical &                                                                                            
  Physical Health                                                                                      

                    Diabetes Diagnosis                                                  Dichotomous    1

                    Multidimensional Fatigue Inventory                                  Ordinal        26
                    [@smetsMultidimensionalFatigueInventory1995]                                       

                    Berlin Sleep Questionnaire [@netzerUsingBerlinQuestionnaire1999]    Ordinal,       3
                                                                                        Dichotomous    

                    Body Mass Index                                                     Numeric        1

                    Health-related Quality of Life Scale                                Ordinal,       4
                    [@taylorMeasuringHealthyDays2000]                                   Numeric        

                    Healthy Days Symptoms Module [@moriartyCDCStudiesCommunity1996]     Numeric        5

  Miscellaneous                                                                                        

                    Treatment                                                           Categorical    1

                    Incarcerated                                                        Dichotomous    1

                    Works Third Shift                                                   Dichotomous    1
  ---------------------------------------------------------------------------------------------------------------


Table 1, formatted for Typst/PDF output

In [ ]:
#| label: tbl-featurespdf
#| tbl-cap: "Individual Differences"

t1_feat_demo |>
  mutate(feat_cat = " ") |>
  rename(`Demographic Characteristics` = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `# Features` = n_feat) |>
tail(-1) |>
  knitr::kable(align = "lllc") 

  ------------------------------------------------------------------------
  Demographic               Feature Name  Type                      \#
  Characteristics                                                Features
  ------------------------- ------------- --------------------- ----------
                            Gender        Categorical               1
                                          (unordered)           

                            Age           Numeric                   1

                            Race          Categorical               1
                                          (unordered)           

                            Marital       Categorical               1
                            Status        (unordered)           

                            Income        Ordinal                   1

                            Ethnicity     Categorical               1
                                          (Unordered)           

                            Employment    Categorical               1
                                          (unordered)           
  ------------------------------------------------------------------------


In [ ]:
#| label: featurespdf2

t1_feat_smoking |>
  mutate(feat_cat = " ") |>
  rename(`Smoking Use/History` = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `# Features` = n_feat) |>
tail(-1) |>
  knitr::kable(align = "lllc") 

  -----------------------------------------------------------------------------------------------------------
  Smoking       Feature Name                                                        Type               \#
  Use/History                                                                                       Features
  ------------- ------------------------------------------------------------------- -------------- ----------
                Baseline Carbon Monoxide                                            Numeric            1

                Carbon Monoxide Exposure                                            Dichotomous        1

                Age of 1st Cigarette                                                Numeric            1

                Age Became Daily Smoker                                             Numeric            1

                Years Smoking                                                       Numeric            1

                Cigarettes Per Day (Heaviest)                                       Numeric            1

                Use of Other Tobacco Products                                       Categorical        5
                                                                                    (Unordered)    

                Number of Previous Quit Attempts                                    Numeric            1

                Last Recent Quit Attempt                                            Ordinal            1

                Longest Quit Attempt                                                Ordinal            1

                Previous Quit Methods Used                                          Dichotomous        6

                Cigarettes Per Day (Current)                                        Numeric            1

                Motivation to Quit                                                  Ordinal            1

                Self-Efficacy for Quitting in Next 30 Days                          Ordinal            1

                Confidence to Quit                                                  Ordinal            1

                Importance to Quit                                                  Ordinal            1

                DSM5 Tobacco Use Disorder                                           Dichotomous        14
                [@americanpsychiatricassociationDiagnosticStatisticalManual2013c]                  

                Fagerstrom Test of Nicotine Dependence                              Dichotomous,       6
                [@heathertonFagerstromTestNicotine1991]                             Ordinal        

                Wisconsin Inventory of Smoking Dependence Motives-37                Ordinal            49
                [@smithDevelopmentBriefWisconsin2010]                                              

                Smoke Menthol Cigarettes                                            Categorical        1
                                                                                    (Unordered)    

                Wisconsin Smoking Withdrawal Scale-2                                Ordinal            45
                [@smithRevisionWisconsinSmoking2021]                                               
  -----------------------------------------------------------------------------------------------------------


In [ ]:
#| label: featurespdf3

t1_feat_social_environ |>
  mutate(feat_cat = " ") |>
  rename(`Social & Environmental Characteristics` = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `# Features` = n_feat) |>
tail(-1) |>
  knitr::kable(align = "lllc") 

  ---------------------------------------------------------------------------
  Social & Environmental     Feature Name         Type                 \#
  Characteristics                                                   Features
  -------------------------- -------------------- ---------------- ----------
                             Spouse Smokes        Categorical          1
                                                  (unordered)      

                             Live with Another    Categorical          1
                             Smoker               (unordered)      

                             People Close to You  Categorical          5
                             Who Smoke            (unordered)      

                             Ban on Smoking at    Dichotomous          1
                             Home                                  

                             Ban on Smoking at    Categorical          1
                             Work                 (unordered)      

                             Time Around Other    Ordinal              2
                             Smokers                               
  ---------------------------------------------------------------------------


  -------------------------------------------------------------------------------------
  Mental Health &     Feature Name                             Type              \#
  Psychological                                                               Features
  Traits                                                                     
  ------------------- ---------------------------------------- ------------- ----------
                      Frequency of Drinking Alcohol            Ordinal           1

                      Quantity of Alcohol                      Ordinal           1

                      Binge Drinking                           Ordinal           1

                      Short Inventory of Problems-2 (Revised)  Ordinal           16
                      [@kilukShortInventoryProblems2013]                     

                      Life Satisfaction                        Ordinal           1

                      Life Enjoyment                           Ordinal           1

                      Psychological Disorder Diagnoses         Dichotomous       7

                      Positive and Negative Affect Schedule    Ordinal           8
                      [@crawfordPositiveNegativeAffect2004a]                 

                      Snaith-Hamilton Pleasure Scale           Ordinal           15
                      [@snaithScaleAssessmentHedonic1995a]                   

                      Anxiety Severity Index-3                 Ordinal           22
                      [@taylorRobustDimensionsAnxiety2007a]                  

                      Distress Tolerance Scale                 Ordinal           20
                      [@simonsDistressToleranceScale2005a]                   

                      Patient History Questionnaire-9          Ordinal           11
                      [@kroenkePHQ9ValidityBrief2001a]                       
  -------------------------------------------------------------------------------------


In [ ]:
#| label: featurespdf4

t1_feat_medical |>
  mutate(feat_cat = " ") |>
  rename(`Medical & Physical Health` = feat_cat,
         `Feature Name` = feat_name,
         Type = type,
         `# Features` = n_feat) |>
tail(-1) |>
  knitr::kable(align = "lllc") 

  -------------------------------------------------------------------------------------
  Medical &     Feature Name                                   Type              \#
  Physical                                                                    Features
  Health                                                                     
  ------------- ---------------------------------------------- ------------- ----------
                Diabetes Diagnosis                             Dichotomous       1

                Multidimensional Fatigue Inventory             Ordinal           26
                [@smetsMultidimensionalFatigueInventory1995]                 

                Berlin Sleep Questionnaire                     Ordinal,          3
                [@netzerUsingBerlinQuestionnaire1999]          Dichotomous   

                Body Mass Index                                Numeric           1

                Health-related Quality of Life Scale           Ordinal,          4
                [@taylorMeasuringHealthyDays2000]              Numeric       

                Healthy Days Symptoms Module                   Numeric           5
                [@moriartyCDCStudiesCommunity1996]                           
  -------------------------------------------------------------------------------------


  Miscellaneous Features   Feature Name        Type           \# Features
  ------------------------ ------------------- ------------- -------------
                           Treatment           Categorical         1
                           Incarcerated        Dichotomous         1
                           Works Third Shift   Dichotomous         1


### Table 2: Demographic Characteristics

Select down data

In [ ]:
dt2 <- d |> 
  select(employment, ends_with("_ehr"), income_order) |> 
  select(-bmi_ehr, -diabetic_ehr) |> 
  glimpse()

Rows: 1,086
Columns: 7
$ employment         <chr> "unemployed", "employed_full_time", "employed_full_…
$ gender_ehr         <chr> "female", "female", "female", "female", "female", "…
$ age_ehr            <dbl> 49, 38, 51, 55, 65, 50, 48, 47, 45, 47, 52, 70, 48,…
$ race_ehr           <chr> "white", "white", "white", "white", "white", "white…
$ marital_status_ehr <chr> "divorced", "married", "divorced", "married", "marr…
$ ethnicity_ehr      <chr> "non_hispanic", "non_hispanic", "non_hispanic", "no…
$ income_order       <chr> "01_less_than_10000", "07_more_than_75000", "06_btw…

Build table

In [ ]:
tab_race <- dt2 |> 
  tab(race_ehr) |> 
  mutate(race_ehr = case_match(
    race_ehr,
    "asian" ~ "   Asian",
    "black_african_american" ~ "   Black/African American",
    "multiracial" ~ "   Multiracial",
    "native_american_alaska_native" ~ "   Native American/Alaska Native",
    "other" ~ "   Other",
    "white" ~ "   White"
  )) |> 
  rename(var_name = race_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Race",
          n_prop = "",
          .before = 1)

tab_ethnicity <- dt2 |> 
  tab(ethnicity_ehr) |> 
  mutate(ethnicity_ehr = case_match(
    ethnicity_ehr,
    "hispanic_latinx" ~ "   Hispanic or Latino/a",
    "non_hispanic" ~ "   Non-Hispanic"
  )) |> 
  rename(var_name = ethnicity_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Ethnicity",
          n_prop = "",
          .before = 1)

tab_gender <- dt2 |> 
  tab(gender_ehr) |> 
  mutate(gender_ehr = case_match(
    gender_ehr,
    "female" ~ "   Female",
    "male" ~ "   Male"
  )) |> 
  rename(var_name = gender_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Gender",
          n_prop = "",
          .before = 1)

tab_employment <- dt2 |> 
  tab(employment) |> 
  mutate(employment = case_match(
    employment,
    "employed_full_time" ~ "   Employed (full-time)",
    "employed_part_time" ~ "   Employed (part-time)",
    "unemployed" ~ "   Unemployed"
  )) |> 
  rename(var_name = employment) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Employment",
          n_prop = "",
          .before = 1)

tab_marital <- dt2 |> 
  tab(marital_status_ehr) |> 
  mutate(marital_status_ehr = case_match(
    marital_status_ehr,
    "divorced" ~ "   Divorced",
    "live_with_domestic_partner" ~ "   Living with a domestic partner",
    "married" ~ "   Married",
    "separated" ~ "   Separated",
    "widowed" ~ "   Widowed",
    "never_married" ~ "   Never married",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = marital_status_ehr) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Marital Status",
          n_prop = "",
          .before = 1)

tab_income <- dt2 |> 
  tab(income_order) |> 
  mutate(income_order = case_match(
    income_order,
    "01_less_than_10000" ~ "   < $10,000",
    "02_btwn_10000_19999" ~ "   $10,000 - $19,999",
    "03_btwn_20000_24999" ~ "   $20,000 - $24,999",
    "04_btwn_25000_34999" ~ "   $25,000 - $34,999",
    "05_btwn_35000_49999" ~ "   $35,000 - $49,999",
    "06_btwn_50000_74999" ~ "   $50,000 - $74,999",
    "07_more_than_75000" ~ "   $75,000+",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = income_order) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Income",
          n_prop = "",
          .before = 1)

tab_age <- tibble(
  var_name = "Age",
  n_prop = "",
  `Mean (SD)` = str_c(round(mean(dt2$age_ehr), digits = 2),
                      " (", round(sd(dt2$age_ehr), digits = 2), ")")
)

t2_demo <- bind_rows(tab_gender, tab_race) |> 
  bind_rows(tab_ethnicity) |> 
  bind_rows(tab_marital) |> 
  bind_rows(tab_employment) |> 
  bind_rows(tab_income) |> 
  mutate(`Mean (SD)` = "") |> 
  add_row(tab_age, .before = 1) |> 
  rename(Characteristic = var_name,
         `N (%)` = n_prop)

Table 2

In [ ]:
#| label: tbl-demohtml
#| tbl-cap: Demographic Characteristics

t2_demo |>
  knitr::kable()

  Characteristic                   N (%)           Mean (SD)
  -------------------------------- --------------- --------------
  Age                                              48.13 (11.6)
  Gender                                           
  Female                           566 (52.12%)    
  Male                             520 (47.88%)    
  Race                                             
  Asian                            3 (0.28%)       
  Black/African American           309 (28.45%)    
  Multiracial                      22 (2.03%)      
  Native American/Alaska Native    6 (0.55%)       
  Other                            18 (1.66%)      
  White                            728 (67.03%)    
  Ethnicity                                        
  Hispanic or Latino/a             27 (2.49%)      
  Non-Hispanic                     1059 (97.51%)   
  Marital Status                                   
  Divorced                         224 (20.63%)    
  Living with a domestic partner   87 (8.01%)      
  Married                          384 (35.36%)    
  Never married                    299 (27.53%)    
  Separated                        53 (4.88%)      
  Widowed                          34 (3.13%)      
  Did not respond                  5 (0.46%)       
  Employment                                       
  Employed (full-time)             474 (43.65%)    
  Employed (part-time)             283 (26.06%)    
  Unemployed                       329 (30.29%)    
  Income                                           
  \< \$10,000                      210 (19.34%)    
  \$10,000 - \$19,999              135 (12.43%)    
  \$20,000 - \$24,999              83 (7.64%)      
  \$25,000 - \$34,999              129 (11.88%)    
  \$35,000 - \$49,999              149 (13.72%)    
  \$50,000 - \$74,999              160 (14.73%)    
  \$75,000+                        167 (15.38%)    
  Did not respond                  53 (4.88%)      


In [ ]:
#| label: tbl-demopdf
#| tbl-cap: Demographic Characteristics

t2_demo |>
  head(-9) |>
  knitr::kable()

  Characteristic                   N (%)           Mean (SD)
  -------------------------------- --------------- --------------
  Age                                              48.13 (11.6)
  Gender                                           
  Female                           566 (52.12%)    
  Male                             520 (47.88%)    
  Race                                             
  Asian                            3 (0.28%)       
  Black/African American           309 (28.45%)    
  Multiracial                      22 (2.03%)      
  Native American/Alaska Native    6 (0.55%)       
  Other                            18 (1.66%)      
  White                            728 (67.03%)    
  Ethnicity                                        
  Hispanic or Latino/a             27 (2.49%)      
  Non-Hispanic                     1059 (97.51%)   
  Marital Status                                   
  Divorced                         224 (20.63%)    
  Living with a domestic partner   87 (8.01%)      
  Married                          384 (35.36%)    
  Never married                    299 (27.53%)    
  Separated                        53 (4.88%)      
  Widowed                          34 (3.13%)      
  Did not respond                  5 (0.46%)       
  Employment                                       
  Employed (full-time)             474 (43.65%)    
  Employed (part-time)             283 (26.06%)    
  Unemployed                       329 (30.29%)    


In [ ]:
#| label: demopdf

t2_demo |>
  tail(9) |>
  knitr::kable()

  Characteristic        N (%)          Mean (SD)
  --------------------- -------------- -----------
  Income                               
  \< \$10,000           210 (19.34%)   
  \$10,000 - \$19,999   135 (12.43%)   
  \$20,000 - \$24,999   83 (7.64%)     
  \$25,000 - \$34,999   129 (11.88%)   
  \$35,000 - \$49,999   149 (13.72%)   
  \$50,000 - \$74,999   160 (14.73%)   
  \$75,000+             167 (15.38%)   
  Did not respond       53 (4.88%)     


### Table 3: Smoking-related ID Characteristics

Select down data

In [ ]:
dt3 <- d |> 
  select(age_1st_cig, age_daily_smoker, years_smoke, cigs_per_day_heaviest,
         tot_quit_attempt, cigs_per_day, dsm5_total_scale, ftnd_1_item_order,
         wisdm37_total_scale, wsws_total_scale)

Build table

In [ ]:
tab_ftnd_1 <- dt3 |> 
  tab(ftnd_1_item_order) |> 
  mutate(ftnd_1_item_order = case_match(
    ftnd_1_item_order,
    "03_within_5_min" ~ "   Within 5 minutes",
    "02_6_to_30_min" ~ "   6 - 30 minutes",
    "01_31_to_60_min" ~ "   31 - 60 minutes",
    "00_after_60_min" ~ "   After 60 minutes",
    NA ~ "   Did not respond"
  )) |> 
  rename(var_name = ftnd_1_item_order) |> 
  mutate(n_prop = str_c(n, " (", round((prop * 100), digits = 2), "%)")) |> 
  select(var_name, n_prop) |> 
  add_row(var_name = "Time to first cigarette upon waking",
          n_prop = "",
          .before = 1) |> 
  rename(`N (%)` = n_prop, Characteristic = var_name) |> 
  mutate(`Mean (SD)` = "")

tab3_means <- dt3 |> 
  select(-ftnd_1_item_order) |> 
  summarize(across(everything(), ~ mean(.x, na.rm = TRUE))) |> 
  pivot_longer(
    cols = everything(),
    names_to = "var_name",
    values_to = "means"
  ) |> 
  mutate(means = round(means, digits = 2))

tab3_sds <- dt3 |> 
  select(-ftnd_1_item_order) |> 
  summarize(across(everything(), ~ sd(.x, na.rm = TRUE))) |> 
  pivot_longer(
    cols = everything(),
    names_to = "var_name",
    values_to = "sds"
  ) |> 
  mutate(sds = round(sds, digits = 2))

t3_idchars <- full_join(tab3_means, tab3_sds, by = "var_name") |> 
  mutate(var_name = case_match(
    var_name,
    "age_1st_cig" ~ "Age of first cigarette",
    "age_daily_smoker" ~ "Age became daily smoker",
    "years_smoke" ~ "Years smoking",
    "cigs_per_day" ~ "Cigarettes per day (current)",
    "cigs_per_day_heaviest" ~ "Cigarettes per day (heaviest)",
    "tot_quit_attempt" ~ "Number of previous quit attempts",
    "dsm5_total_scale" ~ "Number of DSM5 tobacco use disorder symptoms",
    "wisdm37_total_scale" ~ "WISDM37 Total Score",
    "wsws_total_scale" ~ "WSWS Total Score"
  )) |> 
  mutate(`Mean (SD)` = str_c(means, " (", sds, ")"),
         `N (%)` = "") |> 
  rename(Characteristic = var_name) |> 
  select(-means, -sds) |> 
  full_join(tab_ftnd_1,
            by = join_by(Characteristic, `Mean (SD)`, `N (%)`))

Table 3

In [ ]:
#| label: tbl-smoking-chars
#| tbl-cap: Smoking-related ID Characteristics

t3_idchars |>
  knitr::kable()

  -----------------------------------------------------------------------
  Characteristic                               Mean (SD)     N (%)
  -------------------------------------------- ------------- ------------
  Age of first cigarette                       14.6 (3.42)   

  Age became daily smoker                      17.56 (4.62)  

  Years smoking                                28.65 (12.03) 

  Cigarettes per day (heaviest)                22.74 (10.67) 

  Number of previous quit attempts             3.91 (6.03)   

  Cigarettes per day (current)                 17.03 (8.31)  

  Number of DSM5 tobacco use disorder symptoms 4.57 (2.17)   

  WISDM37 Total Score                          3.99 (1.17)   

  WSWS Total Score                             48.82 (19.91) 

  Time to first cigarette upon waking                        

  After 60 minutes                                           82 (7.55%)

  31 - 60 minutes                                            157 (14.46%)

  6 - 30 minutes                                             477 (43.92%)

  Within 5 minutes                                           366 (33.7%)

  Did not respond                                            4 (0.37%)
  -----------------------------------------------------------------------


## Tables 4, 5, & 6 (Treatment Interactions)

Read in data

In [ ]:
model_tidy <- read_csv(file.path(path_models, "model_best_tidy_v6.csv"),
                       show_col_types = FALSE) |> 
  glimpse()

Rows: 1,173
Columns: 3
$ term     <chr> "(Intercept)", "age_ehr", "co", "income_order", "age_1st_cig"…
$ estimate <dbl> 0.749187531, 0.000000000, 0.182416896, -0.055715240, 0.000000…
$ penalty  <dbl> 0.1140801, 0.1140801, 0.1140801, 0.1140801, 0.1140801, 0.1140…

Make overall starting table

In [ ]:
retained_vars_tx <- model_tidy |> 
  mutate(estimate = estimate * -1) |> 
  filter(abs(estimate) > 0) |> 
  select(-penalty) |> 
  filter(str_detect(term, "treatment_")) |> 
  arrange(desc(abs(estimate)))

print_kbl(retained_vars_tx, digits = 4)

term,estimate
treatment_combo_nrt_x_live_with_smoker_live_alone_or_only_with_partner,0.1076
treatment_varenicline_x_live_with_smoker_live_alone_or_only_with_partner,-0.0722
treatment_combo_nrt_x_race_ehr_black_african_american,-0.0663
treatment_combo_nrt_x_wisdm37_22_item_order,-0.0607
treatment_combo_nrt_x_smoke_menthol_no,0.0468
treatment_patch_x_used_cigars_never_tried,-0.0428
treatment_patch_x_close_smoke_friend_no,-0.0419
treatment_combo_nrt_x_dts_5_item_order,0.0415
treatment_varenicline_x_asi3_15_item_order,0.0381
treatment_patch_x_used_cigars_tried,0.0354


### Table 4: Varenicline Interactions

Make table

In [ ]:
varenicline <- retained_vars_tx |> 
  filter(str_detect(term, "varenicline")) |> 
  mutate(term = str_replace(term, "treatment_varenicline_x_", "")) |> 
  arrange(desc(abs(estimate))) |> 
  mutate(direction = if_else(estimate > 0, "+", "-")) |> 
  mutate(Feature = c(
    "Lives alone or only with a partner",
    "Worries they will choke to death when their throat feels tight (ASI-3 Item 15)",
    "Does not think they do a lot in a day (MFI Item 6)",
    "Greater income",
    "Has a spouse/partner who does not smoke",
    "No close relative who smokes",
    "Kept smoking despite relationship problems (DSM-5 Item 3)",
    "Has never tried cigars",
    "Has a depression diagnosis",
    "Does not live with another smoker",
    "Smoking helps them think better (WISDM-37 Item 32)",
    "Feels tired, fatigued, or not up to par (Berlin Item 7)",
    "Has tried cigars but has never used regularly",
    "A lot of friends or family members smoke (WISDM-37 Item 27)",
    "Has a close relative who smokes",
    "Bothered by constipation (WSWS-2 Item 'Constipation')",
    "Smoked to get rid of withdrawal feelings (DSM-5 Item 7)",
    "Tried nicotine gum during previous quit attempt",
    "Has a spouse/partner who smokes",
    "Has not tried nicotine gum during a previous quit attempt",
    "Feeling hopeless or discouraged (WSWS-2 Item 19)",
    "Most of the people they spend time with are smokers (WISDM-37 Item 18)",
    "Doesn't live with a spouse/partner",
    "Smoking only allowed in some parts of workplace"
  )) |> 
  select(Feature, `Effect Direction` = direction)

In [ ]:
#| label: tbl-int-varenicline
#| tbl-cap-location: bottom
#| tbl-cap: "Retained features that interact with varenicline. Direction of effect indicates whether that feature increases or decreases abstinence when using varenicline (compared to C-NRT and nicotine patch)."

varenicline |> 
  knitr::kable(align = "lc")

  -----------------------------------------------------------------------
  Feature                                                      Effect
                                                              Direction
  --------------------------------------------------------- -------------
  Lives alone or only with a partner                             \-

  Worries they will choke to death when their throat feels       \+
  tight (ASI-3 Item 15)                                     

  Does not think they do a lot in a day (MFI Item 6)             \+

  Greater income                                                 \+

  Has a spouse/partner who does not smoke                        \+

  No close relative who smokes                                   \-

  Kept smoking despite relationship problems (DSM-5 Item 3)      \+

  Has never tried cigars                                         \+

  Has a depression diagnosis                                     \+

  Does not live with another smoker                              \+

  Smoking helps them think better (WISDM-37 Item 32)             \-

  Feels tired, fatigued, or not up to par (Berlin Item 7)        \-

  Has tried cigars but has never used regularly                  \-

  A lot of friends or family members smoke (WISDM-37 Item        \-
  27)                                                       

  Has a close relative who smokes                                \+

  Bothered by constipation (WSWS-2 Item 'Constipation')          \+

  Smoked to get rid of withdrawal feelings (DSM-5 Item 7)        \-

  Tried nicotine gum during previous quit attempt                \+

  Has a spouse/partner who smokes                                \-

  Has not tried nicotine gum during a previous quit attempt      \-

  Feeling hopeless or discouraged (WSWS-2 Item 19)               \-

  Most of the people they spend time with are smokers            \-
  (WISDM-37 Item 18)                                        

  Doesn't live with a spouse/partner                             \-

  Smoking only allowed in some parts of workplace                \-
  -----------------------------------------------------------------------


### Table 5: Combo-NRT Interactions

Make table

In [ ]:
combo_nrt <- retained_vars_tx |> 
  filter(str_detect(term, "combo_nrt")) |> 
  mutate(term = str_replace(term, "treatment_combo_nrt_x_", "")) |> 
  arrange(desc(abs(estimate))) |> 
  mutate(direction = if_else(estimate > 0, "+", "-")) |> 
  mutate(Feature = c(
    "Lives alone or only with a partner",
    "Identifies as Black or African American",
    "Sometimes feels like cigarattes are their best friends (WISDM-37 Item 22)",
    "Has never smoked menthol cigarettes",
    "Disagrees that there is nothing worse than feeling distressed or upset (DTS Item 5)",
    "Most of their friends and acquaintances smoke (WISDM-37 Item 30)",
    "Divorced",
    "Does not endorse withdrawal symptoms (DSM-5 Item 6)",
    "Worries they are going crazy when they cannot keep their mind on a task (ASI-3 Item 2)",
    "Identifies as White",
    "Married",
    "Has spent a lot of time obtaining, smoking, or recovering from cigarettes (DSM-5 Item 10)",
    "Currently smokes menthol cigarettes",
    "Endorses weight control as a major reason they smoke (WISDM-37 Item 19)",
    "Does not feel it takes a lot of effort to concentrate (MFI Item 13)",
    "Does not smoke when so ill they are in bed most of the day (FTND Item 6)",
    "Lives with another smoker",
    "More satisfied with their life as a whole",
    "Bothered by constipation (WSWS-2 Item 'Constipation')",
    "Has tried a pipe but has never used regularly",
    "Amount of time since most recent quit attempt",
    "Can handle feeling distressed or upset (DTS Item 3)",
    "Smokes when so ill they are in bed most of the day (FTND Item 6)",
    "Has never been diagnosed with sleep apnea (Berlin Item 9)",
    "Bothered by waking frequently during the night (WSWS-2 Item 5)",
    "Bothered by having urges to smoke (WSWS-2 Item 9)",
    "More days in past month feeling worried, tense, or anxious (HDSM Item `Tense`)",
    "Bothered by wanting to smoke (WSWS-2 Item 'Want Cigarette')",
    "Kept smoking knowing it was causing or worsening a health problem (DSM-5 Item 12)",
    "Does not find feeling distressed or upset to be unbearable (DTS Item 1)",
    "Tried Zyban (bupropion) during previous attempt"
  )) |> 
  select(Feature, `Effect Direction` = direction)

In [ ]:
#| label: tbl-int-cnrt
#| tbl-cap-location: bottom
#| tbl-cap: "Retained features that interact with combination nicotine replacement therapy (C-NRT). Direction of effect indicates whether that feature increases or decreases abstinence when using C-NRT (compared to varenicline and nicotine patch)."

combo_nrt |> 
  knitr::kable(align = "lc")

  -----------------------------------------------------------------------
  Feature                                                       Effect
                                                               Direction
  ----------------------------------------------------------- -----------
  Lives alone or only with a partner                              \+

  Identifies as Black or African American                         \-

  Sometimes feels like cigarattes are their best friends          \-
  (WISDM-37 Item 22)                                          

  Has never smoked menthol cigarettes                             \+

  Disagrees that there is nothing worse than feeling              \+
  distressed or upset (DTS Item 5)                            

  Most of their friends and acquaintances smoke (WISDM-37         \-
  Item 30)                                                    

  Divorced                                                        \-

  Does not endorse withdrawal symptoms (DSM-5 Item 6)             \+

  Worries they are going crazy when they cannot keep their        \-
  mind on a task (ASI-3 Item 2)                               

  Identifies as White                                             \+

  Married                                                         \+

  Has spent a lot of time obtaining, smoking, or recovering       \-
  from cigarettes (DSM-5 Item 10)                             

  Currently smokes menthol cigarettes                             \-

  Endorses weight control as a major reason they smoke            \-
  (WISDM-37 Item 19)                                          

  Does not feel it takes a lot of effort to concentrate (MFI      \+
  Item 13)                                                    

  Does not smoke when so ill they are in bed most of the day      \+
  (FTND Item 6)                                               

  Lives with another smoker                                       \-

  More satisfied with their life as a whole                       \+

  Bothered by constipation (WSWS-2 Item 'Constipation')           \-

  Has tried a pipe but has never used regularly                   \-

  Amount of time since most recent quit attempt                   \+

  Can handle feeling distressed or upset (DTS Item 3)             \+

  Smokes when so ill they are in bed most of the day (FTND        \-
  Item 6)                                                     

  Has never been diagnosed with sleep apnea (Berlin Item 9)       \+

  Bothered by waking frequently during the night (WSWS-2 Item     \+
  5)                                                          

  Bothered by having urges to smoke (WSWS-2 Item 9)               \-

  More days in past month feeling worried, tense, or anxious      \-
  (HDSM Item `Tense`)                                         

  Bothered by wanting to smoke (WSWS-2 Item 'Want Cigarette')     \-

  Kept smoking knowing it was causing or worsening a health       \+
  problem (DSM-5 Item 12)                                     

  Does not find feeling distressed or upset to be unbearable      \+
  (DTS Item 1)                                                

  Tried Zyban (bupropion) during previous attempt                 \-
  -----------------------------------------------------------------------


### Table 6: Patch Interactions

Make table

In [ ]:
patch <- retained_vars_tx |> 
  filter(str_detect(term, "patch")) |> 
  mutate(term = str_replace(term, "treatment_patch_x_", "")) |> 
  arrange(desc(abs(estimate))) |> 
  mutate(direction = if_else(estimate > 0, "+", "-")) |> 
  mutate(Feature = c(
    "Has never tried cigars",
    "No close friend who smokes",
    "Has tried cigars but has never used regularly",
    "Has a close co-worker who smokes",
    "No smoking allowed anywhere at their workplace",
    "Bothered by craving a cigarette (WSWS-2 Item `Crave`)",
    "Endorses feeling alone without their cigarettes (WISDM-37 Item 26)",
    "Has a close friend who smokes",
    "Has never been married",
    "Identifies as female",
    "Finds it difficult to refrain from smoking where it is forbidden (FTND Item 2)",
    "Endorses feeling that cigarettes rule their life (WISDM-37 Item 16)",
    "Identifies as White",
    "Does not work outside the home",
    "Bothered by coughing (WSWS-2 Item `Coughing`)",
    "Does not have a lot of plans (MFI Item 15)",
    "Identifies as male",
    "Bothered by having trouble getting cigarettes of their mind (WSWS-2 Item 26)",
    "Greater total number of previous quit attempts"
  )) |> 
  select(Feature, `Effect Direction` = direction)

In [ ]:
#| label: tbl-int-patch
#| tbl-cap-location: bottom
#| tbl-cap: "Retained features that interact with nicotine patch. Direction of effect indicates whether that feature increases or decreases abstinence when using nicotine patch (compared to varenicline and C-NRT)."

patch |> 
  knitr::kable(align = "lc")

  -----------------------------------------------------------------------
  Feature                                                      Effect
                                                              Direction
  --------------------------------------------------------- -------------
  Has never tried cigars                                         \-

  No close friend who smokes                                     \-

  Has tried cigars but has never used regularly                  \+

  Has a close co-worker who smokes                               \+

  No smoking allowed anywhere at their workplace                 \-

  Bothered by craving a cigarette (WSWS-2 Item `Crave`)          \-

  Endorses feeling alone without their cigarettes (WISDM-37      \+
  Item 26)                                                  

  Has a close friend who smokes                                  \+

  Has never been married                                         \+

  Identifies as female                                           \-

  Finds it difficult to refrain from smoking where it is         \+
  forbidden (FTND Item 2)                                   

  Endorses feeling that cigarettes rule their life               \+
  (WISDM-37 Item 16)                                        

  Identifies as White                                            \+

  Does not work outside the home                                 \+

  Bothered by coughing (WSWS-2 Item `Coughing`)                  \+

  Does not have a lot of plans (MFI Item 15)                     \+

  Identifies as male                                             \+

  Bothered by having trouble getting cigarettes of their         \-
  mind (WSWS-2 Item 26)                                     

  Greater total number of previous quit attempts                 \+
  -----------------------------------------------------------------------
